# Redis LangChain OpenAI eCommerce Chatbot

In [1]:
# Install requirements
!pip install -r ../alchemist/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.3/426.3 kB 6.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.6/238.6 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 19.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 19.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 3.5 MB/s eta 0:00:00ta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.8/157.8 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.1/120.1 kB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 28.0 MB

In [2]:
# Download the dataset
!gdown --id 1tHWB6u3yQCuAgOYc-DxtZ8Mru3uV5_lj

/opt/conda/lib/python3.11/site-packages/gdown/cli.py:126: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (uriginal): https://drive.google.com/uc?id=1tHWB6u3yQCuAgOYc-DxtZ8Mru3uV5_lj
From (redirected): https://drive.google.com/uc?id=1tHWB6u3yQCuAgOYc-DxtZ8Mru3uV5_lj&confirm=t&uuid=a275d655-58b6-454e-9b30-b1335ca1a610
To: /home/jovyan/chatbot/product_data.csv
100%|████████████████████████████████████████| 225M/225M [00:21<00:00, 10.7MB/s]


## Preprocess dataset

In [3]:
import pandas as pd

MAX_TEXT_LENGTH=512

def auto_truncate(val):
    """Truncate the given text."""
    return val[:MAX_TEXT_LENGTH]

# Load Product data and truncate long text fields
all_prods_df = pd.read_csv("product_data.csv", converters={
    'bullet_point': auto_truncate,
    'item_keywords': auto_truncate,
    'item_name': auto_truncate
})

In [7]:
# all_prods_df.head()

In [5]:
all_prods_df.columns

Index(['item_id', 'marketplace', 'country', 'main_image_id', 'domain_name',
       'bullet_point', 'item_keywords', 'material', 'brand', 'color',
       'item_name', 'model_name', 'model_number', 'product_type'],
      dtype='object')

In [6]:
# Contruct a primary key from item ID and domain name
all_prods_df['primary_key'] = (
    all_prods_df['item_id'] + '-' + all_prods_df['domain_name']
)
# Replace empty strings with None and drop
all_prods_df['item_keywords'].replace('', None, inplace=True)
all_prods_df.dropna(subset=['item_keywords'], inplace=True)

# Reset pandas dataframe index
all_prods_df.reset_index(drop=True, inplace=True)

all_prods_df.head()

,item_id,marketplace,country,main_image_id,domain_name,bullet_point,item_keywords,material,brand,color,item_name,model_name,model_number,product_type,primary_key
0,B07T6RZ2CM,Amazon,IN,71dZhpsferL,amazon.in,3D Printed Hard Back Case Mobile Cover for Len...,mobile cover back cover mobile case phone case...,NaN,Amazon Brand - Solimo,Others,Amazon Brand - Solimo Designer Couples Sitting...,Lenovo K4 Note,gz8115-SL40423,CELLULAR_PHONE_CASE,B07T6RZ2CM-amazon.in
1,B07T2JY31Y,Amazon,IN,71vX7qIEAIL,amazon.in,3D Printed Hard Back Case Mobile Cover for Son...,mobile cover back cover mobile case phone case...,Wood,Amazon Brand - Solimo,others,Amazon Brand - Solimo Designer Leaf on Wood 3D...,Sony Xperia Z1 L39H,gz8056-SL40528,CELLULAR_PHONE_CASE,B07T2JY31Y-amazon.in
2,B0849YGSCZ,Amazon,AE,A1EZF-2mB5L,amazon.ae,,small de fur rooms navidad woven girls shag pa...,NaN,Stone & Beam,NaN,Stone & Beam Contemporary Doily Wool Farmhouse...,NaN,I59I8044IVYGRYC00-Parent,HOME_FURNITURE_AND_DECOR,B0849YGSCZ-amazon.ae
3,B081K6TCML,Amazon,IN,81o9EyZ-fAL,amazon.in,Solimo Plastic Multipurpose Modular Drawer; sm...,drawer modular drawer 3 rack modular drawer ki...,Plastic,Amazon Brand - Solimo,Multicolor,Amazon Brand - Solimo Plastic Multipurpose Mod...,NaN,sol_cujo_13,HOME,B081K6TCML-amazon.in
4,B0854774X5,Amazon,IN,81xaJCVnl3L,amazon.in,"Snug fit for Nokia 8.1, with perfect cut-outs ...",Back Cover Designer Case Designer Take It Easy...,Silicon,Amazon Brand - Solimo,Multicolor,Amazon Brand - Solimo Designer Take It Easy UV...,Nokia 8.1,UV10714-SL40617,CELLULAR_PHONE_CASE,B0854774X5-amazon.in


In [8]:
# Num products to use (subset)
NUMBER_PRODUCTS = 2500  

# Get the first 1000 products with non-empty item keywords
product_metadata = ( 
    all_prods_df
     .head(NUMBER_PRODUCTS)
     .to_dict(orient='index')
)

In [13]:
type(product_metadata)

dict

In [9]:
# Check one of the products
product_metadata[0]

{'item_id': 'B07T6RZ2CM',
 'marketplace': 'Amazon',
 'country': 'IN',
 'main_image_id': '71dZhpsferL',
 'domain_name': 'amazon.in',
 'bullet_point': '3D Printed Hard Back Case Mobile Cover for Lenovo K4 Note Easy to put & take off with perfect cutouts for volume buttons, audio & charging ports. Stylish design and appearance, express your unique personality. Extreme precision design allows easy access to all buttons and ports while featuring raised bezel to life screen and camera off flat surface. Slim Hard Back Cover No Warranty None',
 'item_keywords': 'mobile cover back cover mobile case phone case mobile panel phone panel Lenovo mobile case Lenovo phone cover Lenovo back case hard case 3D printed mobile cover mobile cover back cover mobile case phone case mobile panel phone panel Lenovo mobile case Lenovo phone cover Lenovo back case hard case 3D printed mobile cover mobile cover back cover mobile case phone case mobile panel phone panel Lenovo mobile case Lenovo phone cover Lenovo 

## Set up Redis as a vector db

In [14]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores.redis import Redis as RedisVectorStore

# data that will be embedded and converted to vectors
texts = [
    v['item_name'] for k, v in product_metadata.items()
]

# product metadata that we'll store along our vectors
metadatas = list(product_metadata.values())

# we will use OpenAI as our embeddings provider
embedding = OpenAIEmbeddings()

# name of the Redis search index to create
index_name = "products"

# assumes you have a redis stack server running on within your docker compose network
redis_url = "redis://redis:6379"

# create and load redis with documents
vectorstore = RedisVectorStore.from_texts(
    texts=texts,
    metadatas=metadatas,
    embedding=embedding,
    index_name=index_name,
    redis_url=redis_url
)

## Build the ChatBot with ConversationalRetrieverChain

In [15]:
from langchain.callbacks.base import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import (
    ConversationalRetrievalChain,
    LLMChain
)
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from langchain.prompts.prompt import PromptTemplate

template = """Given the following chat history and a follow up question, rephrase the follow up input question to be a standalone question.
Or end the conversation if it seems like it's done.

Chat History:\"""
{chat_history}
\"""

Follow Up Input: \"""
{question}
\"""

Standalone question:"""

condense_question_prompt = PromptTemplate.from_template(template)

template = """You are a friendly, conversational retail shopping assistant. Use the following context including product names, descriptions, and keywords to show the shopper whats available, help find what they want, and answer any questions.
It's ok if you don't know the answer.

Context:\"""
{context}
\"""

Question:\"
\"""

Helpful Answer:"""

qa_prompt= PromptTemplate.from_template(template)


# define two LLM models from OpenAI
llm = OpenAI(temperature=0)

streaming_llm = OpenAI(
    streaming=True,
    callback_manager=CallbackManager([
        StreamingStdOutCallbackHandler()]),
    verbose=True,
    temperature=0.2,
    max_tokens=150
)

# use the LLM Chain to create a question creation chain
question_generator = LLMChain(
    llm=llm,
    prompt=condense_question_prompt
)

# use the streaming LLM to create a question answering chain
doc_chain = load_qa_chain(
    llm=streaming_llm,
    chain_type="stuff",
    prompt=qa_prompt
)


chatbot = ConversationalRetrievalChain(
    retriever=vectorstore.as_retriever(),
    combine_docs_chain=doc_chain,
    question_generator=question_generator
)

In [16]:
# create a chat history buffer
chat_history = []

# gather user input for the first question to kick off the bot
question = input("Hi! What are you looking for today?")

# keep the bot running in a loop to simulate a conversation
while True:
    result = chatbot(
        {"question": question, "chat_history": chat_history}
    )
    print("\n")
    chat_history.append((result["question"], result["answer"]))
    question = input()

Hi! What are you looking for today? gold-plated earrings



Hi there! It sounds like you're looking for some beautiful earrings. We have a few options available that feature Swarovski Zirconia and Swarovski Topaz Gemstones. We have Yellow Gold-Plated Sterling Silver Swarovski Zirconia Fancy Green Stud Earrings, Platinum or Gold-Plated Sterling Silver Princess-Cut Swarovski Zirconia Hoop Earrings, Yellow Gold Plated Sterling Silver Antique Drop Earrings set with Asscher Cut Swarovski Zirconia, and Yellow Gold Plated Sterling Silver Honey Topaz Stud Earrings made with Swarovski Topaz Gemstones. Is there anything else I can help you with?



 My preference is the yellow gold plated sterling silver


 Hi there! I'm here to help you find the perfect earrings. We have a few different options available in yellow gold-plated sterling silver with Swarovski Zirconia. We have the Fancy Green Stud Earrings, the Honey Topaz Stud Earrings made with Swarovski Topaz Gemstones, and the Antique Drop Earrings set with Asscher Cut Swarovski Zirconia. We also have Platinum or Gold-Plated Sterling Silver Princess-Cut Swarovski Zirconia Hoop Earrings. Is there anything else I can help you with?



 no thanks 


...

 A durable iPhone case


 Hi there! I see you're looking for a mobile cover for your Apple iPhone 8 Plus, 11 Pro, or 7 Plus. We have a few options from the Amazon Brand - Solimo Designer line. We have the Sky Blue and Orange Canvas 3D Printed Hard Back Case Mobile Cover for the iPhone 8 Plus, the Universe Printed Hard Back Case Mobile Cover for the iPhone 8 Plus / 7 Plus, and the Abstract 3D Printed Hard Back Case Mobile Cover for the iPhone 11 Pro. All of these cases come with a logo cut. Let me know if you have any questions or need help finding something else.



 no thank you !


 Hi there! I see you're looking for a mobile cover for your 10.or G or Oppo A1K. We have a few options from our Amazon Brand - Solimo Designer line. For the 10.or G, we have the Rose Photography UV Printed Soft Back Case Mobile Cover and the Wooden Beach UV Printed Soft Back Case Mobile Cover. For the Oppo A1K, we have the Chinnese Yin and Yang UV Printed Soft Back Case Mobile Cover and the Blue Pattern Alphabet-S 3D Printed Hard Back Case Mobile Cover. Let me know if you have any questions or need help deciding which one is right for you.



KeyboardInterrupt: Interrupted by user

## Customize your chains for even better performance

In [17]:
import json

from langchain.schema import BaseRetriever
from langchain.vectorstores import VectorStore
from langchain.schema import Document
from pydantic import BaseModel


class RedisProductRetriever(BaseRetriever, BaseModel):
    vectorstore: VectorStore

    class Config:
        
        arbitrary_types_allowed = True

    def combine_metadata(self, doc) -> str:
        metadata = doc.metadata
        return (
            "Item Name: " + metadata["item_name"] + ". " +
            "Item Description: " + metadata["bullet_point"] + ". " +
            "Item Keywords: " + metadata["item_keywords"] + "."
        )

    def get_relevant_documents(self, query):
        docs = []
        for doc in self.vectorstore.similarity_search(query):
            content = self.combine_metadata(doc)
            docs.append(Document(
                page_content=content,
                metadata=doc.metadata
            ))
        return docs

### Setup ChatBot with new retriever

In [18]:
redis_product_retriever = RedisProductRetriever(vectorstore=vectorstore)

chatbot = ConversationalRetrievalChain(
    retriever=redis_product_retriever,
    combine_docs_chain=doc_chain,
    question_generator=question_generator
)

### Retry

In [19]:
# create a chat history buffer
chat_history = []

# gather user input for the first question to kick off the bot
question = input("Hi! What are you looking for today?")

# keep the bot running in a loop to simulate a conversation
while True:
    result = chatbot(
        {"question": question, "chat_history": chat_history}
    )
    print("\n")
    chat_history.append((result["question"], result["answer"]))
    question = input()

Hi! What are you looking for today? iphone cover


 Hi there! We have a few great options for cases for your Apple iPhone 5 / 5S. We have the Amazon Brand - Solimo Designer Alone UV Printed Soft Back Case Mobile Cover, the Amazon Brand - Solimo Designer Photography UV Printed Soft Back Case Mobile Cover, the Amazon Brand - Solimo Designer Believe Printed Hard Back Case Mobile Cover for Apple iPhone 8/7, and the Amazon Brand - Solimo Designer Universe Printed Hard Back Case Mobile Cover for Apple iPhone 8 Plus / 7 Plus. All of these cases feature a snug fit, perfect cutouts for volume buttons, audio and charging ports, stylish designs, and extreme precision design for easy access to all buttons and ports. Let me know if you have any questions about these cases or if you



 thank you please close this converstation




Hi there! I'm here to help you find the perfect item for your needs. Let's start with the AmazonBasics External Case Black Black 24-Pack. This case is perfect for storing or transporting smaller portable hard drives. It has a slimline design that allows it to easily fit into any backpack or briefcase. It also has an interior strap and zippered closure to secure the portable hard drive in place and an internal mesh pocket to store power cords. 

Next, we have the Amazon Brand - Solimo Designer Two Different Patterns 3D Printed Hard Back Case Mobile Cover for Huawei P9 lite. This 3D printed hard back case is easy to put on and take off with perfect cutouts for volume buttons



KeyboardInterrupt: Interrupted by user